# Наблюдатель (observer)

Наблюдатель — это поведенческий паттерн проектирования, который создаёт механизм подписки, позволяющий одним объектам следить и реагировать на события, происходящие в других объектах.


## Пример

Вы участвуете в реализации почтового сервера. Требуется реализовать pushup-уведомление пользователю для двух событий: при поступлении нового персонального письма, при упоминании в рассылке. Эти два типа уведомлений следует обрабатывать по-разному.


In [22]:
from __future__ import annotations
import abc
from typing import Set
from random import randint


class AbsObserver:
    @abc.abstractmethod
    def update(self, subject_item: AbsSubject) -> None:
        ...


class AbsSubject(abc.ABC):
    observers: Set[AbsObserver]

    def __init__(self) -> None:
        self.observers = set()

    def attach(self, observer: AbsObserver) -> None:
        self.observers |= {observer}

    def detach(self, observer: AbsObserver) -> None:
        self.observers -= {observer}

    def notify(self) -> None:
        for observer in self.observers:
            observer.update(self)


class EmailManager(AbsSubject):
    def __init__(self) -> None:
        super().__init__()
        self._received_messages = 0
        self._mentions_in_newsletters = 0

    @property
    def received_messages(self) -> int:
        return self._received_messages

    @property
    def mentions_in_newsletters(self) -> int:
        return self._mentions_in_newsletters

    def synchronize_changes(self) -> None:
        """
        В этом метода происходит синхронизация (получение) изменений с email-сервера.
        """

        some_changes = randint(0, 1)
        if not some_changes:
            print("Нет никаких обновлений на почтовом сервере")
            print()
            return

        self._received_messages += randint(1, 10)
        self._mentions_in_newsletters += randint(1, 5)
        self.notify()


class OnReceivedNotifier(AbsObserver):
    def update(self, subject_item: EmailManager) -> None:
        val = subject_item.received_messages
        print("[ВСПЛЫВАЮЩЕЕ УВЕДОМЛЕНИЕ СВЕРХУ]")
        print(f"Вам пришло новое письмо. Всего: {val}")
        print()


class OnMentionedNotifier(AbsObserver):
    def update(self, subject_item: EmailManager) -> None:
        val = subject_item.mentions_in_newsletters
        print("[ВСПЛЫВАЮЩЕЕ УВЕДОМЛЕНИЕ СНИЗУ]")
        print(f"Вас упомянули в рассылке. Всего: {val}")
        print()

In [33]:
email_manager = EmailManager()
notifier_received = OnReceivedNotifier()
email_manager.attach(notifier_received)
notifier_mentioned = OnMentionedNotifier()
email_manager.attach(notifier_mentioned)

# Вероятность, что письмо придет 0.5, т. ч. вызываем метод `synchronize_changes` несколько раз
email_manager.synchronize_changes()
email_manager.synchronize_changes()

Нет никаких обновлений на почтовом сервере

[ВСПЛЫВАЮЩЕЕ УВЕДОМЛЕНИЕ СВЕРХУ]
Вам пришло новое письмо. Всего: 9

[ВСПЛЫВАЮЩЕЕ УВЕДОМЛЕНИЕ СНИЗУ]
Вас упомянули в рассылке. Всего: 2

